# I. Loading/Plot functions

In [27]:
df_fix_mu = df[df['mu']==.2] # Currently mu in [.05,.1,.2,.4]
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                           sem            \
method           dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K name                                                                         
2 AWA           15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.000693  0.001058   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003954  0.002942   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001567  0.002324   
  dogs           NaN   5.0   5.0   NaN   NaN   5.0   5.0       NaN  0.004987   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002848  0.002848   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.016838  0.013397   
  plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001480  0.001172   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005031  0.004786   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.011326  0.015278   
4 AWA            5.0   5.0  10.0  10.0  10.0  10.0  10.0  0.000792  0.000740   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002447  0.003824   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000218  0.000588   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.001564   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001638  0.003804   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007205  0.005376   
  plt-net        2.0   3.0   5.0   5.0   5.0   NaN   5.0  0.000134  0.001930   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.010090  0.003310   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008621  0.007312   
8 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001424  0.000955   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000475  0.000865   
  birds          NaN   5.0   5.0   5.0   5.0  10.0  10.0       NaN  0.000182   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.001722   
  medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.001738  0.002140   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   NaN  0.002201  0.002359   
  plt-net        NaN   5.0   5.0   2.0   3.0   5.0   5.0       NaN  0.000943   
  resisc         3.0   NaN   5.0   5.0   5.0   5.0   5.0  0.005392       NaN   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006883  0.001137   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.000912  0.000895  0.000819  0.000248  0.001803  
  airplanes     0.006266  0.005733  0.004522  0.009099  0.003539  
  birds         0.004976  0.001816  0.003265  0.005415  0.003679  
  dogs          0.005396       NaN       NaN  0.006555  0.010999  
  medical-leaf  0.012948  0.002848  0.002326  0.005696  0.006780  
  plt-doc       0.010416  0.013045  0.012522  0.029757  0.015031  
  plt-net       0.002058  0.001649  0.001978  0.001249  0.001473  
  resisc        0.001867  0.003463  0.003421  0.006568  0.003623  
  texture-dtd   0.020880  0.008391  0.014797  0.014467  0.024611  
4 AWA           0.000920  0.000482  0.000560  0.000340  0.000537  
  airplanes     0.003036  0.004953  0.004302  0.003883  0.003374  
  birds         0.000390  0.000369  0.000275  0.000677  0.000906  
  dogs          0.001277       NaN       NaN       NaN  0.002622  
  medical-leaf  0.003374  0.001176  0.002518  0.012856  0.010980  
  plt-doc       0.008013  0.006064  0.003139  0.006292  0.006162  
  plt-net       0.000767  0.000766  0.002559       NaN  0.000735  
  resisc        0.001645  0.002409  0.005807  0.005984  0.004261  
  texture-dtd   0.003446  0.010546  0.011076  0.009626  0.023341  
8 AWA           0.001680  0.001051  0.000725  0.001132  0.001015  
  airplanes     0.001167  0.001968  0.000638  0.001062  0.000650  
  birds         0.000181  0.000130  0.000363  0.000610  0.000130  
  dogs

In [14]:
DATASETS = {
    'a': 'dogs',
    'b': 'medical-leaf',
    'c': 'texture-dtd',
    'd': 'birds',
    'e': 'AWA',
    'f': 'plt-net',
    'g': 'resisc',
    'h': 'plt-doc',
    'i': 'airplanes',
    '_': 'ALL'
}

color_dict = {
        'erm': 'blue',
        'jtt': 'red',
        'suby': '#00CC96',  # green
        'subg': '#ff7f0e',  # orange
        'rwy': '#00CC96',
        'rwg': '#ff7f0e',
        'dro': '#DEA0FD'  # purple
    }
    

In [9]:
import wandb
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Initialize wandb
wandb.login()

# Set your entity and project
entity_name = "aureliengauffre"  # e.g., your username or team name
project_name = "SMA_all_2_best"

# Initialize the wandb API ||client
api = wandb.Api()

# Fetch all runs from the specified project
runs = api.runs(f"{entity_name}/{project_name}")

# Create an empty list to hold data for each run
data = []

# Loop through runs and ext/ract the data you're interested in
for run in runs:
    # Extract both summary metrics and config (hyperparameters) for each run
    run_data = {
        "name": run.name,
        "summary_metrics": run.summary._json_dict,
        "config": run.config,
        # Add any other attributes you're interested in here
    }
    data.append(run_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(data)

# For summary metrics and config (hyperparameters), expand them into separate columns
df_summary = pd.json_normalize(df['summary_metrics'])
df_config = pd.json_normalize(df['config'])
df = pd.concat([df.drop(['summary_metrics', 'config'], axis=1), df_summary, df_config], axis=1)
df = df.iloc[:, 1:]  # Drop the first column containing the name of the runs (since there is also another name column)

# Now we have a DataFrame `df` with all runs, their summary metrics, and hyperparameters
# print(df.head())  # Print the first few rows of the DataFrame


In [15]:
df['name']

0            plt-net
1            plt-net
2            plt-net
3            plt-net
4            plt-net
            ...     
3307       airplanes
3308       airplanes
3309    medical-leaf
3310       airplanes
3311       airplanes
Name: name, Length: 3312, dtype: object

In [16]:
# Initialize a Plotly figure
def plot_graph_old(df, x_axis, y_axis, dataset_name, methods=None):
    # Default methods if not provided
    
    if methods is None:
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if dataset_name is not None:
        df_dataset = df[(df['name'] == dataset_name) ]#& (df['mu'] >= .1) ]
    else :
        df_dataset = df 
    fig = go.Figure() # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean of y_axis
        df_avg = df_method.groupby(x_axis)[y_axis].mean().reset_index()
        
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line to the plot for the current method
        color = color_dict.get(method, 'grey')
        fig.add_trace(go.Scatter(x=df_avg[x_axis], y=df_avg[y_axis], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis} , dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=600   # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

# Example usage (assuming df is your DataFrame):
# plot_graph(df, 'K', 'mean_gr


dataset_name = 'AWA' #73sports
y_axis = 'mean_grp_acc_te' #'mean_grp_acc_te'
x_axis = 'K'
plot_graph(df, x_axis, y_axis, dataset_name, methods=None)


In [17]:
import plotly.graph_objects as go


def normalize_df(df,metric = 'best_acc_te'):
    # Copy the DataFrame to avoid modifying the original data
    result_df = df.copy()
    
    # Group by 'name', 'K', 'mu', and 'init_seed' and calculate the minimum and maximum accuracy
    grouped = df.groupby(['name', 'K', 'mu', 'init_seed'])[metric]
    min_acc = grouped.transform(np.min)
    max_acc = grouped.transform(np.max)
    
    # Apply the Min-Max normalization formula
    result_df[metric] = (df[metric] - min_acc) / (max_acc - min_acc)
    
    return result_df




def plot_graph(df, x_axis, y_axis, dataset_name, error_bars=None, normalize = False, methods=None,):
    print(dataset_name)
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    color_dict = {
            'erm': 'blue',
            'jtt': 'red',
            'suby': '#00CC96',  # green
            'subg': '#ff7f0e',  # orange
            'rwy': '#00CC96',
            'rwg': '#ff7f0e',
            'dro': '#DEA0FD'  # purple
        }
    if dataset_name is None or dataset_name == 'ALL': 
        df_dataset = df
    else:
        df_dataset = df[(df['name'] == dataset_name)]
    
    if normalize :
        df_dataset = normalize_df(df_dataset,metric=y_axis)
        
    fig = go.Figure()  # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean, standard deviation, and count (for standard error calculation)
        stats = df_method.groupby(x_axis)[y_axis].agg(['mean', 'std', 'count']).reset_index()
        
        # Calculate standard error (SEM)
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        #print(method, stats) # print the number on which we average, interesting !
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color),
                                 error_y=dict(type='data', array=stats['sem'], visible=True)))
        else :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    
    fig.show()


def plot_graph_all(df, x_axis, y_axis, error_bars=None, normalize=False, methods=None):
    """Unlike the original plot_graph function which calculates and plots error bars
    based on individual datasets directly, this version computes the standard error within each dataset first 
    and then averages these errors across all datasets for each method. This approach provides a generalized 
    view of method performance and variability across different datasets."""
    
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if normalize:
        df = normalize_df(df, metric=y_axis)

    fig = go.Figure()  # Initialize a Plotly figure
    all_stats = pd.DataFrame()

    # Process each method separately
    for method in methods:
        df_method = df[df['method'] == method]

        # Group data first by 'name' and then by x_axis and compute statistics
        grouped = df_method.groupby(['name', x_axis])
        stats = grouped[y_axis].agg(['mean', 'std', 'count']).reset_index()

        # Calculate standard error within each dataset
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        
        # Append results to the all_stats DataFrame for later display
        stats['method'] = method
        all_stats = pd.concat([all_stats, stats], ignore_index=True)
        
        # Now group by x_axis and calculate the mean of the means and the mean of the SEMs
        final_stats = stats.groupby(x_axis).agg({'mean': 'mean', 'sem': 'mean'}).reset_index()

        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color),
                                     error_y=dict(type='data', array=final_stats['sem'], visible=True)))
        else:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color)))

    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, Average on all dataset (mean and std)',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

    # Display the all_stats DataFrame
    return all_stats.pivot_table(index=[x_axis, 'name'], columns='method', values=['count', 'sem'], aggfunc='first')



# II. Analysis : K

Here are the **plot parameters** to be played with :

In [18]:
ERROR_BARS = True # Wether to plot the error bars
NORMALIZE = False

To analyse the impact of K, we the **value of mu is fixed**:

In [19]:

df_fix_mu_K2 = df[df['K']==2] 
df_fix_mu_K4 = df[df['K']==4] 
df_fix_mu_K8 = df[df['K']==8] 
df_fix_mu_K12 = df[df['K']==12] 

## a. best_acc_te

In [22]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                      sem            \
method           dro  erm  jtt  rwg  rwy  subg suby       dro       erm   
K name                                                                    
2 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.004747  0.004054   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.026757  0.022423   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.009868  0.008473   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.035725  0.018371   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.016197  0.013920   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.030264  0.016465   
  plt-net        NaN  NaN  5.0  5.0  5.0   5.0  5.0       NaN       NaN   
  resisc         5.0  NaN  NaN  5.0  5.0   5.0  NaN  0.002964       NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.011744  0.019270   
4 AWA            5.0  5.0  5.0  5.0  5.0  10.0  5.0  0.003190  0.003885   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.009539  0.010725   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.001275  0.001531   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.011016  0.006940   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.011374  0.015285   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.003576  0.011876   
  plt-net        NaN  NaN  5.0  4.0  5.0   NaN  3.0       NaN       NaN   
  resisc         5.0  NaN  NaN  5.0  5.0   5.0  NaN  0.020028       NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.011940  0.016236   
8 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.002992  0.002323   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.010417  0.002784   
  birds          NaN  5.0  NaN  0.0  5.0   5.0  5.0       NaN  0.000648   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.003001  0.003588   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.003857  0.012974   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.002929  0.006227   
  plt-net        2.0  5.0  5.0  5.0  5.0   5.0  5.0  0.003887  0.001055   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.009783  0.006837   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.007132  0.006483  0.010450  0.001766  0.005871  
  airplanes     0.031813  0.033598  0.013061  0.010423  0.010402  
  birds         0.015478  0.010897  0.009312  0.009032  0.017364  
  dogs          0.020373  0.018918  0.007196  0.017807  0.015543  
  medical-leaf  0.031995  0.009443  0.014640  0.036265  0.029514  
  plt-doc       0.024901  0.016794  0.007456  0.016770  0.010886  
  plt-net       0.009891  0.009055  0.012012  0.004271  0.007995  
  resisc             NaN  0.017274  0.007656  0.022949       NaN  
  texture-dtd   0.010440  0.012917  0.009160  0.045728  0.021916  
4 AWA           0.005705  0.005085  0.005435  0.004109  0.002976  
  airplanes     0.007702  0.010926  0.004434  0.009269  0.008506  
  birds         0.001989  0.001680  0.000884  0.000622  0.001915  
  dogs          0.008785  0.006708  0.005702  0.010057  0.016070  
  medical-leaf  0.009660  0.018760  0.008245  0.021000  0.027321  
  plt-doc       0.013196  0.002780  0.006386  0.010614  0.006044  
  plt-net       0.004128  0.004247  0.004035       NaN  0.009946  
  resisc             NaN  0.005776  0.004534  0.003151       NaN  
  texture-dtd   0.011970  0.007345  0.011122  0.011970  0.014827  
8 AWA           0.002195  0.003131  0.005197  0.001333  0.005126  
  airplanes     0.001780  0.002696  0.003501  0.004118  0.006740  
  birds              NaN       NaN  0.000500  0.000399  0.000303  
  dogs          0.003823  0.003697  0.006104  0.003655  0.006033  
  medical-leaf  0.024009  0.007222  0.009421  0.011260  0.014054  
  plt-doc       0.004943  0.003293  0.004953  0.004478  0.009267  
  plt-net       0.002272  0.00

In [ ]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [ ]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                      sem             \
method           dro  erm  jtt  rwg  rwy  subg suby       dro        erm   
K name                                                                     
2 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  5.285351   7.815770   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  9.015927   4.307786   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  5.909254  14.278238   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  1.269381   2.218810   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  1.176471   1.176471   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  2.113973   0.827737   
  plt-net        NaN  NaN  5.0  5.0  5.0   5.0  5.0       NaN        NaN   
  resisc         5.0  NaN  NaN  5.0  5.0   5.0  NaN  1.452186        NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  8.888889   3.664141   
4 AWA            5.0  5.0  5.0  4.0  5.0  10.0  5.0  4.160248   3.901398   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  8.719997   4.673160   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.902846   0.628074   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  4.800716   1.688531   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  3.463401   6.430667   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.984982   0.863433   
  plt-net        NaN  NaN  5.0  5.0  5.0   NaN  3.0       NaN        NaN   
  resisc         5.0  NaN  NaN  NaN  NaN   5.0  NaN  4.678799        NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  2.576005   1.039349   
8 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  2.913004   2.827365   
  airplanes      NaN  5.0  NaN  NaN  NaN   5.0  5.0       NaN   0.255472   
  birds          NaN  5.0  NaN  0.0  5.0   5.0  5.0       NaN   0.500760   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.677408   0.596249   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  1.412508   1.474230   
  plt-doc        NaN  5.0  NaN  NaN  NaN   NaN  NaN       NaN   2.181480   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.000000   0.000000   

                                                                   
method               jtt        rwg       rwy      subg      suby  
K name                                                             
2 AWA           4.474268   8.141688  4.872016  1.386377  6.253393  
  airplanes     0.927458   6.662022  5.712818  4.630114  1.496619  
  birds         2.968569  10.256449  7.664581  2.671972  5.154132  
  dogs          1.538159   2.567449  1.059739  4.139578  1.644408  
  medical-leaf  1.440876   1.440876  2.352941  5.346400  4.822336  
  plt-doc       2.847008   5.062968  2.510024  2.963089  0.437409  
  plt-net       3.276794   2.996579  5.101449  0.808741  1.061529  
  resisc             NaN   0.968904  2.220294  7.880341       NaN  
  texture-dtd   3.685139   7.954345  6.047650  5.241101  2.421611  
4 AWA           6.215691   4.929448  3.960460  1.488967  4.094905  
  airplanes     3.497415   3.262211  5.015975  3.067890  5.313958  
  birds         1.730366   0.644938  0.467041  1.881298  1.870886  
  dogs          3.973173   1.656076  3.583545  5.916469  1.865240  
  medical-leaf  2.630668   4.962838  3.421142  2.200605  2.261243  
  plt-doc       1.866966   1.611635  1.107338  1.665806  0.976014  
  plt-net       3.138322   2.212506  2.228063       NaN  0.510998  
  resisc             NaN        NaN       NaN  4.043415       NaN  
  texture-dtd   1.842569   2.421611  2.041241  2.078699  1.619709  
8 AWA           2.594125   2.725117  4.078984  3.056242  1.713918  
  airplanes          NaN        NaN       NaN  3.855175  0.140469  
  birds              NaN        NaN  0.829958  0.590586  0.545743  
  dogs          0.865555   1.040422  1.376042  1.885632  1.043435  
  medical-leaf  0.000000   3.628690  1.462056  2.118302  0.000000  
  plt-doc            NaN        NaN       NaN       NaN       NaN  
  texture-dtd   0.000000   0.555556  0.000000  2.4

In [ ]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [ ]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## d. minor_group_acc

In [ ]:
x_axis = 'K'
y_axis = 'minor_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# III. Analysis : mu

Here are the **plot parameters** to be played with :

In [ ]:
ERROR_BARS = True #Wether to plot the error bars
NORMALIZE = True

To analyse the impact of mu, we the **value of K is fixed**:

In [ ]:
df_fix_K = df[df['K']==4] # Currently K in [2,4,8,12]


## a. best_acc_te

In [ ]:
x_axis = 'mu'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [ ]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                      sem            \
method           dro  erm  jtt  rwg  rwy  subg suby       dro       erm   
K name                                                                    
2 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.092018  0.132032   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.153852  0.075890   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.148002  0.181808   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.024087  0.030690   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.028583  0.014470   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.035873  0.026982   
  plt-net        NaN  NaN  5.0  5.0  5.0   5.0  5.0       NaN       NaN   
  resisc         5.0  NaN  NaN  5.0  5.0   5.0  NaN  0.145231       NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.139816  0.055556   
4 AWA            5.0  5.0  5.0  4.0  5.0  10.0  5.0  0.108243  0.077551   
  airplanes      5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.221646  0.163497   
  birds          5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.106451  0.113567   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.139484  0.075740   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.114954  0.089838   
  plt-doc        5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.071216  0.080840   
  plt-net        NaN  NaN  5.0  5.0  5.0   NaN  3.0       NaN       NaN   
  resisc         5.0  NaN  NaN  NaN  NaN   5.0  NaN  0.244949       NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.174136  0.028571   
8 AWA            5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.102515  0.054578   
  airplanes      NaN  5.0  NaN  NaN  NaN   5.0  5.0       NaN  0.024922   
  birds          NaN  5.0  NaN  0.0  5.0   5.0  5.0       NaN  0.117556   
  dogs           5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.048009  0.037613   
  medical-leaf   5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.075835  0.073532   
  plt-doc        NaN  0.0  NaN  NaN  NaN   NaN  NaN       NaN       NaN   
  texture-dtd    5.0  5.0  5.0  5.0  5.0   5.0  5.0  0.000000  0.000000   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.127909  0.109882  0.094693  0.000000  0.126117  
  airplanes     0.013539  0.105813  0.091734  0.080541  0.000000  
  birds         0.132811  0.194167  0.178965  0.147394  0.140133  
  dogs          0.025268  0.049911  0.027393  0.000000  0.016629  
  medical-leaf  0.034319  0.030321  0.036364  0.000000  0.104876  
  plt-doc       0.051883  0.078977  0.035866  0.000000  0.007857  
  plt-net       0.032727  0.089198  0.140957  0.000000  0.018112  
  resisc             NaN  0.168175  0.107412  0.216816       NaN  
  texture-dtd   0.088696  0.113709  0.079679  0.112279  0.036679  
4 AWA           0.086022  0.107780  0.065932  0.021874  0.069323  
  airplanes     0.094765  0.077495  0.088507  0.095966  0.089001  
  birds         0.144277  0.029769  0.039364  0.152823  0.114685  
  dogs          0.054108  0.124285  0.094144  0.018526  0.030303  
  medical-leaf  0.040334  0.100090  0.151309  0.058469  0.061261  
  plt-doc       0.154957  0.122786  0.075370  0.060465  0.037117  
  plt-net       0.136584  0.052370  0.139340       NaN  0.000000  
  resisc             NaN       NaN       NaN  0.244949       NaN  
  texture-dtd   0.094993  0.143372  0.091897  0.085714  0.093953  
8 AWA           0.059629  0.087441  0.109612  0.000000  0.010777  
  airplanes          NaN       NaN       NaN  0.000000  0.000000  
  birds              NaN       NaN  0.196353  0.096075  0.193503  
  dogs          0.044613  0.085468  0.048524  0.020000  0.045569  
  medical-leaf  0.000000  0.105409  0.118821  0.155012  0.000000  
  plt-doc            NaN       NaN       NaN       NaN       NaN  
  texture-dtd   0.000000  0.028571  0.000000  0.000000  0.000000

In [ ]:
x_axis = 'mu'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [ ]:
x_axis = 'mu'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# IV. Examples 

## a. Dataset difficulty :

In [ ]:
def plot_violin(df, metric, category):
    """
    Creates and displays a violin plot for the given DataFrame.
    Parameters:
    - df: pandas DataFrame containing the data to plot.
    - metric: str, the name of the column in df containing the values to plot.
    - category: str, the name of the column in df representing different categories to separate the violins.

    """
    fig = go.Figure()
    categories = df[category].unique()
    for cat in categories:
        cat_data = df[df[category] == cat][metric]
        fig.add_trace(go.Violin(y=cat_data, name=cat, box_visible=True, meanline_visible=True))

    fig.update_layout(title=f"{metric} by {category} for dataset",
                      yaxis_title=metric,
                      legend_title=category)

    # Show the figure
    fig.show()

In [ ]:
plot_violin(df_fix_mu,'best_acc_te','method')

In [ ]:
plot_violin(df_fix_mu_K2,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu_K12,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu,'best_acc_te','name')

In [ ]:
plot_violin(df_fix_mu_K2,'best_acc_te','name')

In [ ]:
plot_violin(df_fix_mu,'worst_grp_acc_te','name')

In [ ]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [ ]:
df_1 = df_fix_mu[df_fix_mu['name'] == 'plt-net']
df_1 = df_1[df_1['K'] == 2]
df_1 = df_1[df_1['method'] == 'erm']
df_1['best_acc_te']

Series([], Name: best_acc_te, dtype: float64)

In [ ]:
df_2 = df_fix_mu[df_fix_mu['name'] == 'resisc']
df_2 = df_2[df_2['K'] == 2]
df_2 = df_2[df_2['method'] == 'erm']
df_2['best_acc_te']

Series([], Name: best_acc_te, dtype: float64)

In [ ]:
df_2['best_acc_te'].std()/(5)**.5 # We check the computation from previous error bar in graphs

nan